In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import preprocessing as kprocessing
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import re
import nltk
import gensim.downloader as api
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

2024-02-13 20:35:26.427100: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-13 20:35:26.685449: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-13 20:35:26.687742: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-13 20:35:27.783298: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/lib/python3/dist-packages/paramiko/transport.py:220: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [28]:
file_path='/content/drive/MyDrive/Colab Notebooks/Topic_summarization_data'
epoch=10
batch_size=128
nr_categories=4
class_names=['World', 'Sports', 'Business', 'Sci/Tech']

In [6]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')

In [7]:
X_train=train_df['text']
X_test=test_df['text']
y_train_class=train_df['label']
y_test_class=test_df['label']

In [8]:
lab = LabelBinarizer()
lab.fit(y_train_class)
y_train = lab.transform(y_train_class)
y_test=lab.transform(y_test_class)

In [9]:
print("y_train:",y_train)
print("y_test:",y_test)

y_train: [[0 0 0 1]
 [0 1 0 0]
 [0 0 1 0]
 ...
 [1 0 0 0]
 [0 1 0 0]
 [0 0 1 0]]
y_test: [[0 1 0 0]
 [1 0 0 0]
 [0 0 0 1]
 ...
 [0 0 0 1]
 [0 0 1 0]
 [0 1 0 0]]


In [10]:
n=101
print(f'Coding of labels into a one-hot vector: {y_train_class[n]} is {y_train[n]}')

Coding of labels into a one-hot vector: 1 is [0 1 0 0]


In [70]:
corpus = X_train
tokenizer = kprocessing.text.Tokenizer(lower=True, split=' ', oov_token="<pad>", filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(corpus)
voc = tokenizer.word_index
reverse_voc = dict([(value, key) for (key, value) in voc.items()])

In [71]:
max_words=len(voc)

In [72]:
max_len = 200
sequences = tokenizer.texts_to_sequences(X_train)
X_train_seq = kprocessing.sequence.pad_sequences(sequences, maxlen=max_len)

In [73]:
sequences_test = tokenizer.texts_to_sequences(X_test)
X_test_seq = kprocessing.sequence.pad_sequences(sequences_test, maxlen=max_len)

In [74]:
X_train_seq

array([[   0,    0,    0, ...,   51,  304, 3159],
       [   0,    0,    0, ..., 2561,  565,   57],
       [   0,    0,    0, ...,   96,   80,   42],
       ...,
       [   0,    0,    0, ..., 2964, 2038, 1542],
       [   0,    0,    0, ..., 2681, 1832,  564],
       [   0,    0,    0, ...,   11,  633,  427]], dtype=int32)

In [75]:
X_test_seq

array([[   0,    0,    0, ...,  650,  495,   58],
       [   0,    0,    0, ...,  465,  188,   43],
       [   0,    0,    0, ...,   82, 1020,   92],
       ...,
       [   0,    0,    0, ..., 6154,  282, 1854],
       [   0,    0,    0, ...,  768, 2968,   73],
       [   0,    0,    0, ..., 7053, 6973,  760]], dtype=int32)

In [76]:
n=200 # You can adjust n
print('Shape: ',X_train_seq.shape)
print(100*'-')
print('Example: ',X_train_seq[n,:])

Shape:  (120000, 200)
----------------------------------------------------------------------------------------------------
Example:  [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0

In [18]:
w2v = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [19]:
w2v['hello'].shape

(300,)

In [77]:
emb_matrix=np.zeros((max_words+1, 300))
for i in range(max_words):
    w = reverse_voc[i+1]
    if w in w2v:
        emb_matrix[i+1,:] = w2v[w]
emb_size = emb_matrix.shape[1]

# BRNN

In [21]:
def brnnModel():
  input_ = layers.Input(shape=X_train_seq.shape[1:], name='input')
  x = layers.Embedding(max_words + 1, emb_size, weights=[emb_matrix], trainable=False, name='embedding')(input_)
  x = layers.Bidirectional(layers.SimpleRNN(15, dropout=0.2, return_sequences=False), name='bidirectional-rnn')(x)  # BRNN layer
  x = layers.Dropout(0.2, name='dropout')(x)
  x = layers.Dense(64, activation='relu', name='dense')(x)
  output = layers.Dense(nr_categories, activation='softmax', name='classification')(x)
  callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=1),  # Stop training if validation loss does not improve for 3 epochs
    ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)  # Save the best model based on validation accuracy
  ]

  # Create the model
  model = models.Model(input_, output)
  return model,callbacks

In [22]:
brnn_model,callbacks=brnnModel()

2024-02-13 09:01:08.092183: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-13 09:01:08.248512: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [78]:
brnn_model2,callbacks=brnnModel()

In [23]:
opt = optimizers.Adam(learning_rate=0.01, beta_1=0.9)
brnn_model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])

In [81]:
opt = optimizers.Adam(learning_rate=0.01, beta_1=0.9)
brnn_model2.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
brnn_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 200)]             0         
                                                                 
 embedding (Embedding)       (None, 200, 300)          6000300   
                                                                 
 bidirectional-rnn (Bidirec  (None, 30)                9480      
 tional)                                                         
                                                                 
 dropout (Dropout)           (None, 30)                0         
                                                                 
 dense (Dense)               (None, 64)                1984      
                                                                 
 classification (Dense)      (None, 4)                 260       
                                                             

In [29]:
history = brnn_model.fit(X_train_seq, y_train, batch_size=batch_size, shuffle=True, epochs=epoch,validation_data=(X_test_seq, y_test),callbacks=callbacks)

Epoch 1/10
938/938 [==============================] - ETA: 0s - loss: 0.7377 - accuracy: 0.7296
Epoch 1: val_accuracy improved from -inf to 0.68776, saving model to best_model.h5
938/938 [==============================] - 164s 175ms/step - loss: 0.7377 - accuracy: 0.7296 - val_loss: 0.7834 - val_accuracy: 0.6878
Epoch 2/10


/home/yuvraj/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


938/938 [==============================] - ETA: 0s - loss: 0.7378 - accuracy: 0.7358
Epoch 2: val_accuracy improved from 0.68776 to 0.77408, saving model to best_model.h5
938/938 [==============================] - 143s 152ms/step - loss: 0.7378 - accuracy: 0.7358 - val_loss: 0.6704 - val_accuracy: 0.7741
Epoch 3/10
938/938 [==============================] - ETA: 0s - loss: 0.7579 - accuracy: 0.7291
Epoch 3: val_accuracy did not improve from 0.77408
938/938 [==============================] - 157s 167ms/step - loss: 0.7579 - accuracy: 0.7291 - val_loss: 0.6656 - val_accuracy: 0.7713
Epoch 4/10
938/938 [==============================] - ETA: 0s - loss: 0.8885 - accuracy: 0.6326
Epoch 4: val_accuracy did not improve from 0.77408
938/938 [==============================] - 145s 155ms/step - loss: 0.8885 - accuracy: 0.6326 - val_loss: 1.1121 - val_accuracy: 0.5167
Epoch 5/10
938/938 [==============================] - ETA: 0s - loss: 1.0977 - accuracy: 0.5198
Epoch 5: val_accuracy did not impr

In [82]:
history2 = brnn_model2.fit(X_train_seq, y_train, batch_size=batch_size, shuffle=True, epochs=epoch,validation_data=(X_test_seq, y_test),callbacks=callbacks)

Epoch 1/10
938/938 [==============================] - ETA: 0s - loss: 0.7381 - accuracy: 0.7308
Epoch 1: val_accuracy improved from -inf to 0.75224, saving model to best_model.h5
938/938 [==============================] - 151s 159ms/step - loss: 0.7381 - accuracy: 0.7308 - val_loss: 0.7358 - val_accuracy: 0.7522
Epoch 2/10


/home/yuvraj/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


938/938 [==============================] - ETA: 0s - loss: 0.7222 - accuracy: 0.7586
Epoch 2: val_accuracy improved from 0.75224 to 0.76803, saving model to best_model.h5
938/938 [==============================] - 155s 165ms/step - loss: 0.7222 - accuracy: 0.7586 - val_loss: 0.6714 - val_accuracy: 0.7680
Epoch 3/10
938/938 [==============================] - ETA: 0s - loss: 0.7833 - accuracy: 0.6814
Epoch 3: val_accuracy did not improve from 0.76803
938/938 [==============================] - 156s 166ms/step - loss: 0.7833 - accuracy: 0.6814 - val_loss: 0.7869 - val_accuracy: 0.6792
Epoch 4/10
938/938 [==============================] - ETA: 0s - loss: 0.7944 - accuracy: 0.6794
Epoch 4: val_accuracy did not improve from 0.76803
938/938 [==============================] - 158s 168ms/step - loss: 0.7944 - accuracy: 0.6794 - val_loss: 0.7961 - val_accuracy: 0.6768
Epoch 5/10
938/938 [==============================] - ETA: 0s - loss: 0.7221 - accuracy: 0.7321
Epoch 5: val_accuracy improved fro

In [30]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import re

In [31]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


False

In [32]:
stopwords = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [33]:
def clean_text(text):
    text=str(text)
    tokens = word_tokenize(text)
    tokens[0]=tokens[0][2:]
    cleaned_text=[lemmatizer.lemmatize(word.lower()) for word in tokens if word.lower() not in stopwords and word not in string.punctuation]
    return ' '.join(cleaned_text)

In [95]:
def predict(text):
  cleaned_text=clean_text(text)
  seq=tokenizer.texts_to_sequences(cleaned_text)
  seq=kprocessing.sequence.pad_sequences(seq, maxlen=max_len)
  pred_seq=np.expand_dims(seq,axis=1)
  prediction=brnn_model.predict(seq)
  print(prediction.shape)
  prediction_per_sent=np.argmax(prediction,axis=1)
  predicted_label=class_names[np.bincount(prediction_per_sent).argmax()]
  print(predicted_label)

In [96]:
predict("In recent years, artificial intelligence (AI) has revolutionized various industries, ranging from healthcare to finance. One of the most notable advancements in AI is the development of deep learning algorithms, which enable machines to learn complex patterns and make intelligent decisions autonomously. Applications of deep learning span diverse domains, including natural language processing (NLP), computer vision, and speech recognition. Companies are leveraging deep learning models to enhance customer experience, optimize business processes, and drive innovation. As the field continues to evolve, researchers are exploring novel architectures and techniques to push the boundaries of AI further. With the rapid advancements in technology and the growing availability of data, the potential for AI to reshape industries and improve human lives is more promising than ever.")

22/22 [==============================] - 0s 15ms/step
(678, 4)
Sci/Tech


In [103]:
predict("down trend")

1/1 [==============================] - 0s 18ms/step
(8, 4)
Sports


# LSTM

In [104]:
def lstmModel():
  input_ = layers.Input(shape=X_train_seq.shape[1:], name='input')
  x = layers.Embedding(max_words + 1, emb_size, weights=[emb_matrix], trainable=False, name='embedding')(input_)
  x = layers.Bidirectional(layers.LSTM(15, dropout=0.2, return_sequences=False), name='bidirectional-rnn')(x)  # BRNN layer
  x = layers.Dropout(0.2, name='dropout')(x)
  x = layers.Dense(64, activation='relu', name='dense')(x)
  output = layers.Dense(nr_categories, activation='softmax', name='classification')(x)
  callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, verbose=1),  # Stop training if validation loss does not improve for 3 epochs
    ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)  # Save the best model based on validation accuracy
  ]

  # Create the model
  model = models.Model(input_, output)
  return model,callbacks

In [105]:
lstm_model,callbacks1=lstmModel()

In [106]:
opt = optimizers.Adam(learning_rate=0.01, beta_1=0.9)
lstm_model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])

In [107]:
history_lstm = lstm_model.fit(X_train_seq, y_train, batch_size=batch_size, shuffle=True, epochs=epoch,validation_data=(X_test_seq, y_test),callbacks=callbacks)

Epoch 1/10
938/938 [==============================] - ETA: 0s - loss: 0.3768 - accuracy: 0.8736
Epoch 1: val_accuracy improved from 0.80118 to 0.89447, saving model to best_model.h5


/home/yuvraj/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


938/938 [==============================] - 228s 239ms/step - loss: 0.3768 - accuracy: 0.8736 - val_loss: 0.3115 - val_accuracy: 0.8945
Epoch 2/10
938/938 [==============================] - ETA: 0s - loss: 0.3178 - accuracy: 0.8933
Epoch 2: val_accuracy improved from 0.89447 to 0.89842, saving model to best_model.h5
938/938 [==============================] - 243s 259ms/step - loss: 0.3178 - accuracy: 0.8933 - val_loss: 0.3042 - val_accuracy: 0.8984
Epoch 3/10
 59/938 [>.............................] - ETA: 3:45 - loss: 0.3031 - accuracy: 0.8987

KeyboardInterrupt: 